# Бейзлайн

## Модель на транзакционных данных

### Импортируем библиотеки

In [10]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier as not_xgb
from sklearn.metrics import f1_score
%matplotlib inline
from matplotlib import pyplot as plt

### Подгружаем данные

In [11]:
df = pd.read_csv('train_data_prepared.csv', sep=';')
df_test = pd.read_csv('test.csv', sep=';')

### Описываем данные

Размеры выборок

In [12]:
print(df.shape)
print(df_test.shape)

(6285, 909)
(351, 907)


В тестовой выборке нет таргета

In [13]:
print(set(df.columns).difference(df_test.columns))

{'period', 'target'}


Количество разных типов фичей

In [14]:
df.dtypes.value_counts()

float64    816
int64       90
object       3
dtype: int64

Рейт первого класса

In [15]:
df.target.value_counts(normalize=True)

0    0.537947
1    0.462053
Name: target, dtype: float64

In [16]:
df.client_id.nunique()

351

351 уникальных клиентов в выборке

### Строим модель

#### Цель: предсказать вероятность дефолта клиента через 12 месяцев на основании его исторических транзакций

Избавляемся от object фичей

In [20]:
print('{:30} {}'.format('column', 'nunique'))
for col in df.dtypes[df.dtypes == 'object'].index:
    print('{:30} {}'.format(col, df[col].nunique()))

column                         nunique
cash_in_out                    2
display_type                   2
scanner_code_reader            2


In [21]:
df = pd.get_dummies(df, drop_first=True, columns=df.dtypes[df.dtypes == 'object'].index)
df_test = pd.get_dummies(df_test, drop_first=True, columns=df_test.dtypes[df_test.dtypes == 'object'].index)

Делим выборку на семпл для построения и валидации

In [23]:
df_train, df_valid = df[df.period < 23], df[df.period == 23]

In [24]:
import lightgbm
from lightgbm import LGBMClassifier
model = LGBMClassifier(
        max_depth=3,
        n_estimators=3000,
        min_child_samples=80,
        subsample=0.8)

model.fit(df_train.drop(['target', 'period'], axis=1)
          , df_train.target
          )

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=3, min_child_samples=80,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.8,
        subsample_for_bin=200000, subsample_freq=1)

Модель предсказывает вероятность, нам нужно выбрать cutoff

In [25]:
df_valid.target.value_counts(normalize=True)

1    0.547009
0    0.452991
Name: target, dtype: float64

In [26]:
cutoff = 0.404

0.404 было подобрано таким образом, чтобы на валидационной выборке рейт класса 1 совпадал

In [27]:
pd.Series(model.predict_proba(df_valid.drop(['target', 'period'], axis=1))[:, 1] > cutoff).astype(int).value_counts(normalize=True)

1    0.581197
0    0.418803
dtype: float64

In [28]:
f1_score(df_valid.target, model.predict_proba(df_valid.drop(['target', 'period'], axis=1))[:, 1] > cutoff)

0.65656565656565657

In [30]:
baseline = pd.Series((model.predict_proba(df_test)[:, 1] > cutoff).astype(int))
baseline = baseline.to_frame().reset_index()
baseline.columns = ['CLIENT_ID', 'PREDICT']

In [32]:
baseline.to_csv('result.csv', index=False)

Качество модели оцениваем по коэффициенту **GINI** - должно быть лучше чем текущая модель **0.60**